# ------------------------------**Sung** Work Begins Here--------------------------------

# Importing from CSV file
## = CDC.gov Dataset =

### Gather dependencies

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
# from sodapy import Socrata
from config import census_key
import pprint
import requests
import os
import scipy.stats as st
from scipy.stats import linregress

### Create DataFrame for CDC Data

In [2]:
cdc_data = pd.read_csv("../Project_1/cdc_data.csv")
cdc_data
cdc_df = pd.DataFrame(cdc_data)
cdc_df

# Choose the columns we want and rename anything that isn't clear
cdc_df = cdc_df[['case_month', 'res_state', 'res_county', 'age_group', 'sex', 'race', 'ethnicity', 'current_status']]
cdc_df = cdc_df.rename(columns={
    'res_state': 'State',
    'res_county': 'County',
    'ethnicity': 'Ethnicity',
    'current_status': 'Infection Status',
    'race':'Race'
})

# Drop "Unkown" race from dataset
cdc_df = cdc_df.loc[cdc_df['Race']!= 'Unknown', :]

# Set Infections to 1 and replace the various races with values that are compatible with the census data
cdc_df['Infections'] = 1
cdc_df['Race'] = cdc_df['Race'].replace('White', 'Caucasian')
cdc_df['Race'] = cdc_df['Race'].replace('Black', 'African American')
cdc_df['Race'] = cdc_df['Race'].replace('American Indian/Alaska Native', 'Native American')
cdc_df['Race'] = cdc_df['Race'].replace('Multiple/Other', 'Other')
cdc_df['Race'] = np.where(np.logical_and(cdc_df['Race'] == 'Caucasian', cdc_df['Ethnicity'] == 'Hispanic/Latino'), 'Hispanic', cdc_df['Race'])

# NOTE: It's not clear what happens with Hawaiian here. 

cdc_df

,case_month,State,County,age_group,sex,Race,Ethnicity,Infection Status,Infections
0,2021-11,MN,STEELE,18 to 49 years,Male,Caucasian,Non-Hispanic/Latino,Laboratory-confirmed case,1
2,2021-11,MN,STEARNS,50 to 64 years,Female,Caucasian,Non-Hispanic/Latino,Laboratory-confirmed case,1
3,2020-05,MN,KANDIYOHI,18 to 49 years,Male,Caucasian,Non-Hispanic/Latino,Laboratory-confirmed case,1
4,2021-05,MN,ANOKA,18 to 49 years,Male,NaN,NaN,Laboratory-confirmed case,1
5,2020-04,MN,CLAY,65+ years,Female,Caucasian,Non-Hispanic/Latino,Laboratory-confirmed case,1
...,...,...,...,...,...,...,...,...,...
539745,2022-01,MN,SCOTT,50 to 64 years,Male,Asian,Non-Hispanic/Latino,Laboratory-confirmed case,1
539746,2020-11,MN,GOODHUE,18 to 49 years,Female,Caucasian,Non-Hispanic/Latino,Laboratory-confirmed case,1
539747,2020-11,MN,DAKOTA,18 to 49 years,Male,Caucasian,Non-Hispanic/Latino,Laboratory-confirmed case,1
539748,2022-01,MN,DAKOTA,50 to 64 years,Male,Caucasian,Non-Hispanic/Latino,Laboratory-confirmed case,1


## Infections by race in MN

In [3]:
# Drop NaNs and then sort by date chronologically
cdc_df = cdc_df.dropna(how='any')
cdc_df = cdc_df[['case_month', 'County', 'Race', 'Infections']]

cdc_df

,case_month,County,Race,Infections
0,2021-11,STEELE,Caucasian,1
2,2021-11,STEARNS,Caucasian,1
3,2020-05,KANDIYOHI,Caucasian,1
5,2020-04,CLAY,Caucasian,1
6,2020-10,NOBLES,Hispanic,1
...,...,...,...,...
539745,2022-01,SCOTT,Asian,1
539746,2020-11,GOODHUE,Caucasian,1
539747,2020-11,DAKOTA,Caucasian,1
539748,2022-01,DAKOTA,Caucasian,1


In [4]:
race_mn = cdc_df['Race'].unique()
race_mn = sorted(set(race_mn))
race_mn

['African American',
 'Asian',
 'Caucasian',
 'Hispanic',
 'Native American',
 'Other']

In [5]:

cdc_date = cdc_df['case_month'].unique()
cdc_date.sort()
print(cdc_date)

['2020-03' '2020-04' '2020-05' '2020-06' '2020-07' '2020-08' '2020-09'
 '2020-10' '2020-11' '2020-12' '2021-01' '2021-02' '2021-03' '2021-04'
 '2021-05' '2021-06' '2021-07' '2021-08' '2021-09' '2021-10' '2021-11'
 '2021-12' '2022-01' '2022-02' '2022-03' '2022-04' '2022-05' '2022-06']


In [6]:
cdc_df = cdc_df.sort_values(['case_month', 'County'])
cdc_df

,case_month,County,Race,Infections
232369,2020-03,ANOKA,Caucasian,1
275945,2020-03,ANOKA,Caucasian,1
305881,2020-03,ANOKA,Caucasian,1
354490,2020-03,ANOKA,Caucasian,1
408125,2020-03,ANOKA,Caucasian,1
...,...,...,...,...
512646,2022-06,WRIGHT,Caucasian,1
528869,2022-06,WRIGHT,Caucasian,1
531029,2022-06,WRIGHT,Caucasian,1
534619,2022-06,WRIGHT,Caucasian,1


## Number of Infections by Race and County
   ### a) Total infections for African American.

In [7]:
#total sum of infections of African American.
cdc_infections_african_american_df = cdc_df.loc[  cdc_df['Race'] == race_mn[0]]
cdc_infections_african_american_df
cdc_infections_african_american_df = cdc_infections_african_american_df.groupby(['County', 'Race'])
cdc_infections_african_american_df.head()
cdc_infections_african_american_sum_df = cdc_infections_african_american_df.sum()
cdc_infections_african_american_sum_df.head(50)

,,Infections
County,Race,
ANOKA,African American,2864
BELTRAMI,African American,3
BENTON,African American,104
BLUE EARTH,African American,213
CARLTON,African American,13
CARVER,African American,82
CHISAGO,African American,28
CLAY,African American,190
DAKOTA,African American,3433


In [8]:
cdc_infections_african_american_total = cdc_infections_african_american_sum_df['Infections'].sum()
cdc_infections_african_american_total

36467

### b) Total infections for Asian.

In [9]:
#total sum of infections of Asians
cdc_infections_asian_df = cdc_df.loc[  cdc_df['Race'] == race_mn[1]]
cdc_infections_asian_df
cdc_infections_asian_df = cdc_infections_asian_df.groupby(['County', 'Race'])
cdc_infections_asian_df.head()
cdc_infections_asian_sum_df = cdc_infections_asian_df.sum()
cdc_infections_asian_sum_df.head(50)

,,Infections
County,Race,
ANOKA,Asian,1732
BLUE EARTH,Asian,30
CARVER,Asian,123
CHISAGO,Asian,14
CLAY,Asian,6
DAKOTA,Asian,1551
FREEBORN,Asian,39
HENNEPIN,Asian,7448
ISANTI,Asian,2


In [10]:
cdc_infections_asian_total = cdc_infections_asian_sum_df['Infections'].sum()
cdc_infections_asian_total

20792

### c) Total infections for Caucasian American.

In [11]:
cdc_infections_caucasian_df = cdc_df.loc[  cdc_df['Race'] == race_mn[2]]
cdc_infections_caucasian_df
cdc_infections_caucasian_df = cdc_infections_caucasian_df.groupby(['County', 'Race'])
cdc_infections_caucasian_df.head()
cdc_infections_caucasian_sum_df = cdc_infections_caucasian_df.sum()
cdc_infections_caucasian_sum_df.head(50)

,,Infections
County,Race,
ANOKA,Caucasian,24501
BECKER,Caucasian,1992
BELTRAMI,Caucasian,2216
BENTON,Caucasian,4164
BLUE EARTH,Caucasian,4555
BROWN,Caucasian,1886
CARLTON,Caucasian,1838
CARVER,Caucasian,7256
CASS,Caucasian,1502


In [12]:
cdc_infections_caucasian_total = cdc_infections_caucasian_sum_df['Infections'].sum()
cdc_infections_caucasian_total

328541

### d) Total infections for Hispanic.

In [13]:
#total sum of infections of asians in 2021
cdc_infections_hispanic_df = cdc_df.loc[  cdc_df['Race'] == race_mn[3]]
cdc_infections_hispanic_df
cdc_infections_hispanic_df = cdc_infections_hispanic_df.groupby(['County', 'Race'])
cdc_infections_hispanic_df.head()
cdc_infections_hispanic_sum_df = cdc_infections_hispanic_df.sum()
cdc_infections_hispanic_sum_df.head(50)

,,Infections
County,Race,
ANOKA,Hispanic,746
BENTON,Hispanic,1
BLUE EARTH,Hispanic,50
BROWN,Hispanic,46
CARVER,Hispanic,179
CHISAGO,Hispanic,10
CLAY,Hispanic,87
DAKOTA,Hispanic,1532
FREEBORN,Hispanic,141


In [14]:
cdc_infections_hispanic_total = cdc_infections_hispanic_sum_df['Infections'].sum()
cdc_infections_hispanic_total

10265

### e) Total infections for Native American.

In [15]:
#total sum of infections of Native American in 2021
cdc_infections_native_american_df = cdc_df.loc[  cdc_df['Race'] == race_mn[4]]
cdc_infections_native_american_df
cdc_infections_native_american_df = cdc_infections_native_american_df.groupby(['County', 'Race'])
cdc_infections_native_american_df.head()
cdc_infections_native_american_sum_df = cdc_infections_native_american_df.sum()
cdc_infections_native_american_sum_df.head(50)

,,Infections
County,Race,
ANOKA,Native American,73
BECKER,Native American,228
BELTRAMI,Native American,1064
CARLTON,Native American,64
CASS,Native American,195
CHISAGO,Native American,2
CLAY,Native American,29
DAKOTA,Native American,65
HENNEPIN,Native American,707


In [16]:
cdc_infections_native_american_total = cdc_infections_native_american_sum_df['Infections'].sum()
cdc_infections_native_american_total

3045

### f) Total infections for other.

In [17]:
#total sum of infections of other in 2021
cdc_infections_other_df = cdc_df.loc[  cdc_df['Race'] == race_mn[5]]
cdc_infections_other_df
cdc_infections_other_df = cdc_infections_other_df.groupby(['County', 'Race'])
cdc_infections_other_df.head()
cdc_infections_other_sum_df = cdc_infections_other_df.sum()
cdc_infections_other_sum_df.head(50)

,,Infections
County,Race,
ANOKA,Other,348
BECKER,Other,2
BELTRAMI,Other,33
BENTON,Other,12
BLUE EARTH,Other,21
CARLTON,Other,6
CARVER,Other,49
CHISAGO,Other,5
CLAY,Other,69


In [18]:
cdc_infections_other_total = cdc_infections_other_sum_df['Infections'].sum()
cdc_infections_other_total

5499

In [19]:
#summary
total_infection_byrace = [cdc_infections_african_american_total, cdc_infections_asian_total, 
                               cdc_infections_caucasian_total, cdc_infections_hispanic_total, 
                               cdc_infections_native_american_total, cdc_infections_other_total]
for (race, infection) in zip(race_mn, total_infection_byrace):
    print(f" {race}: {infection}")

 African American: 36467
 Asian: 20792
 Caucasian: 328541
 Hispanic: 10265
 Native American: 3045
 Other: 5499


In [20]:
african_american_asian_combined_df = pd.merge(cdc_infections_african_american_sum_df, cdc_infections_asian_sum_df, on=['County'], how = "outer")
african_american_asian_caucasian_combined_df = pd.merge(african_american_asian_combined_df, cdc_infections_caucasian_sum_df, on=['County'], how = "outer")
african_american_asian_caucasian_hispanic_combined_df = pd.merge(african_american_asian_caucasian_combined_df, cdc_infections_hispanic_sum_df, on=['County'], how = "outer")
african_american_asian_caucasian_hispanic_native_american_combined_df = pd.merge(african_american_asian_caucasian_hispanic_combined_df, cdc_infections_native_american_sum_df, on=['County'], how = "outer")

C:\Users\Sung\AppData\Local\Temp\ipykernel_7248\37040428.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Infections_x'} in the result is deprecated and will raise a MergeError in a future version.
  african_american_asian_caucasian_hispanic_combined_df = pd.merge(african_american_asian_caucasian_combined_df, cdc_infections_hispanic_sum_df, on=['County'], how = "outer")


In [21]:
# Rename columns. FINAL summary table
comprehensive_df = pd.merge(african_american_asian_caucasian_hispanic_native_american_combined_df, cdc_infections_other_sum_df, on=['County'], how = "outer")
comprehensive_df.columns = ['African American Infections', 'Asian Infections', 'Caucasian American Infections', 'Hispanic Infections', 'Native American Infections', 'Other Infections']
comprehensive_df
# comprehensive_clean_df = comprehensive_df.fillna(0)
comprehensive_df

C:\Users\Sung\AppData\Local\Temp\ipykernel_7248\1390579093.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Infections_x'} in the result is deprecated and will raise a MergeError in a future version.
  comprehensive_df = pd.merge(african_american_asian_caucasian_hispanic_native_american_combined_df, cdc_infections_other_sum_df, on=['County'], how = "outer")


,African American Infections,Asian Infections,Caucasian American Infections,Hispanic Infections,Native American Infections,Other Infections
County,,,,,,
ANOKA,2864.0,1732.0,24501,746.0,73.0,348.0
BELTRAMI,3.0,NaN,2216,NaN,1064.0,33.0
BENTON,104.0,NaN,4164,1.0,NaN,12.0
BLUE EARTH,213.0,30.0,4555,50.0,NaN,21.0
CARLTON,13.0,NaN,1838,NaN,64.0,6.0
CARVER,82.0,123.0,7256,179.0,NaN,49.0
CHISAGO,28.0,14.0,4158,10.0,2.0,5.0
CLAY,190.0,6.0,5152,87.0,29.0,69.0
DAKOTA,3433.0,1551.0,26590,1532.0,65.0,643.0


# Pulling data using API with the help of JSON
## = Census Dataset =

In [22]:
# Function for quickly assempling a DataFrame
def json_to_dataframe(response):
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [23]:
# Poverty DataFrame By Race - 2020

url = "https://api.census.gov/data/2020/acs/acs5?get=NAME,B17001_002E,B17001A_002E,B17001B_002E,B17001C_002E,B17001D_002E,B17001E_002E,B17001F_002E,B17001G_002E,B17001H_002E,B17001I_002E&for=county:*&in=state:27&key={0}".format(census_key)
response = requests.request("GET", url)
response
poverty_df = json_to_dataframe(response)

# Split NAME into county & state
name = poverty_df['NAME'].str.split(",", n=1, expand=True)
COUNTY = poverty_df['County'] = name[0]
STATE = poverty_df['State'] = name[1]
poverty_df.drop(columns=["NAME"], inplace=True)
poverty_df['Caucasian2'] = poverty_df['B17001A_002E'].astype(int) - poverty_df['B17001I_002E'].astype(int)
poverty_df['Other'] = poverty_df['B17001F_002E'].astype(int) + poverty_df['B17001G_002E'].astype(int)

# Get the columns we want
poverty_df = poverty_df[['County', 'State','B17001_002E','Caucasian2', 'B17001B_002E','B17001C_002E','B17001D_002E','B17001E_002E','Other','B17001I_002E']]

# Rename columns to something intelligible
poverty_df = poverty_df.rename(columns={
    'B17001_002E':'Poverty Total',
    'Caucasian2':'Caucasian',
    'B17001B_002E':'African American',
    'B17001C_002E':'Native American',
    'B17001D_002E':'Asian',
    'B17001E_002E':'Hawaiian', # I had to change this from Native Hawaiian to Hawaiian for compatibility with the rest - Dom
    'B17001I_002E':'Hispanic' 
    })

# # Remove ...County from County in dataframe
poverty_df['County'] = poverty_df['County'].replace('County', '', regex=True)
# NOTE: The line above accidentally leaves a blank space at the end of these county names! That space caused a merge to fail
# on any of these counties because, for example, "AITKIN " is not the same as "AITKIN". 

# Remove any blank space that may be present at the begining or end of the county name
poverty_df['County'] = poverty_df['County'].str.strip()

# And then convert the county name to uppercase
poverty_df['County'] = poverty_df['County'].str.upper()

poverty_df['Year'] = 2020
#poverty_df.to_csv("../Project_1/population_in_poverty.csv")
poverty_df



,County,State,Poverty Total,Caucasian,African American,Native American,Asian,Hawaiian,Other,Hispanic,Year
0,ANOKA,Minnesota,20270,11778,3697,327,953,0,1964,1551,2020
1,BECKER,Minnesota,3588,2296,17,709,45,0,381,140,2020
2,BELTRAMI,Minnesota,8167,3657,310,3056,39,0,734,371,2020
3,BENTON,Minnesota,3357,2605,333,19,73,0,205,122,2020
4,BIG STONE,Minnesota,596,596,0,0,0,0,0,0,2020
...,...,...,...,...,...,...,...,...,...,...,...
82,RENVILLE,Minnesota,1373,717,29,97,8,0,245,277,2020
83,ROSEAU,Minnesota,1133,971,6,67,11,0,28,50,2020
84,SHERBURNE,Minnesota,4953,2857,578,171,251,0,737,359,2020
85,STEELE,Minnesota,2887,1835,426,113,3,0,149,361,2020


In [24]:
# Choose only the columns we need
poverty_df = poverty_df[['County', 'Caucasian', 'African American', 'Native American', 'Asian', 'Hawaiian', 'Other', 'Hispanic']]
poverty_df

,County,Caucasian,African American,Native American,Asian,Hawaiian,Other,Hispanic
0,ANOKA,11778,3697,327,953,0,1964,1551
1,BECKER,2296,17,709,45,0,381,140
2,BELTRAMI,3657,310,3056,39,0,734,371
3,BENTON,2605,333,19,73,0,205,122
4,BIG STONE,596,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
82,RENVILLE,717,29,97,8,0,245,277
83,ROSEAU,971,6,67,11,0,28,50
84,SHERBURNE,2857,578,171,251,0,737,359
85,STEELE,1835,426,113,3,0,149,361


In [25]:
# Use melt to convert this into a format similar to the CDC data
poverty_df = poverty_df.melt(id_vars='County', value_vars=['Caucasian', 'African American', 'Native American', 'Asian', \
                                                    'Hawaiian', 'Other', 'Hispanic'], \
                                                     var_name='Race', value_name='Poverty')
poverty_gb = poverty_df.groupby(['County', 'Race'])
poverty_df = poverty_gb.sum()
poverty_df.reset_index(inplace=True)
poverty_df

,County,Race,Poverty
0,AITKIN,African American,13
1,AITKIN,Asian,0
2,AITKIN,Caucasian,1711
3,AITKIN,Hawaiian,0
4,AITKIN,Hispanic,45
...,...,...,...
604,YELLOW MEDICINE,Caucasian,722
605,YELLOW MEDICINE,Hawaiian,0
606,YELLOW MEDICINE,Hispanic,53
607,YELLOW MEDICINE,Native American,30


In [26]:
# Population by county, by race
url = "https://api.census.gov/data/2020/acs/acs5?get=NAME,B03002_001E,B03002_003E,B03002_004E,B03002_005E,B03002_006E,B03002_007E,B03002_008E,B03002_009E,B03002_012E&for=county:*&in=state:27&key={0}".format(census_key)


response = requests.request("GET", url)
response
response_df = json_to_dataframe(response)
response_df = response_df.rename(columns={
    'B03002_001E': 'Population Total',
    'B03002_003E': 'Caucasian',
    'B03002_004E': 'African American',
    'B03002_005E': 'Native American',
    'B03002_006E': 'Asian',
    'B03002_007E': 'Hawaiian',
    'B03002_012E': 'Hispanic',
#     'B03002_005E': 'American Indian and Alaska Native alone'
})
response_df['Other'] = response_df['B03002_008E'].astype(int) + response_df['B03002_009E'].astype(int)
mn_pop_df = response_df

# Split NAME into county & state
name = mn_pop_df['NAME'].str.split(",", n=1, expand=True)
COUNTY = mn_pop_df['County']=name[0]
STATE = mn_pop_df['State']=name[1]
mn_pop_df.drop(columns=["NAME"], inplace=True)

# # Remove ...County from County in dataframe
mn_pop_df['County'] = mn_pop_df['County'].replace('County', '', regex=True)
# NOTE: The line above accidentally leaves a blank space at the end of these county names! That space caused a merge to fail
# on any of these counties because, for example, "AITKIN " is not the same as "AITKIN". 

# Remove any blank space that may be present at the begining or end of the county name
mn_pop_df['County'] = mn_pop_df['County'].str.strip()

# # Upper case for County
mn_pop_df['County'] = mn_pop_df['County'].str.upper()
mn_pop_df = mn_pop_df[['County', 'Population Total', 'Caucasian', 'African American', 'Native American', 'Asian', 'Hawaiian', 'Hispanic', 'Other']]
mn_pop_df.to_csv("../Project_1/mn_population by race.csv")
mn_pop_df

,County,Population Total,Caucasian,African American,Native American,Asian,Hawaiian,Hispanic,Other
0,ANOKA,353775,283436,23534,1788,16082,86,16769,12080
1,BECKER,34227,29290,146,1938,209,6,738,1900
2,BELTRAMI,46784,33506,399,8920,295,36,1114,2514
3,BENTON,40476,35979,1876,49,378,0,1086,1108
4,BIG STONE,4974,4787,3,3,0,0,102,79
...,...,...,...,...,...,...,...,...,...
82,RENVILLE,14572,12741,67,186,108,0,1268,202
83,ROSEAU,15259,13876,124,254,399,0,223,383
84,SHERBURNE,96015,86342,2658,403,1206,0,2577,2829
85,STEELE,36710,31640,1225,119,150,6,2913,657


### Create population DataFrame by county and race

In [27]:
# Use melt to convert this into a format similar to the CDC data
pop_df = mn_pop_df.melt(id_vars='County', value_vars=['Caucasian', 'African American', 'Native American', 'Asian', \
                                                      'Hawaiian', 'Other', 'Hispanic'], \
                                                      var_name='Race', value_name='Population')
pop_gb = pop_df.groupby(['County', 'Race'])
pop_df = pop_gb.sum()
pop_df.reset_index(inplace=True)
pop_df

,County,Race,Population
0,AITKIN,African American,61
1,AITKIN,Asian,40
2,AITKIN,Caucasian,14831
3,AITKIN,Hawaiian,9
4,AITKIN,Hispanic,216
...,...,...,...
604,YELLOW MEDICINE,Caucasian,8725
605,YELLOW MEDICINE,Hawaiian,0
606,YELLOW MEDICINE,Hispanic,468
607,YELLOW MEDICINE,Native American,229


In [28]:
#population by race infected with covid-19

mn_pop = mn_pop_df['Population Total']
mn_caucasian_pop = mn_pop_df['Caucasian']
mn_african_american_pop = mn_pop_df['African American']
mn_native_american_pop = mn_pop_df['Native American']
mn_asian_pop = mn_pop_df['Asian']
mn_hawaiian_pop = mn_pop_df['Hawaiian']
mn_hispanic_pop = mn_pop_df['Hispanic']
mn_other_pop = mn_pop_df['Other']

In [29]:
mn_pop_list = list(mn_pop)
mn_pop_list
mn_caucasian_list = list(mn_caucasian_pop)
mn_caucasian_list
mn_african_american_list = list(mn_african_american_pop)
mn_african_american_list
mn_native_american_list = list(mn_native_american_pop)
mn_native_american_list
mn_asian_list = list(mn_asian_pop)
mn_asian_list
mn_hawaiian_list = list(mn_hawaiian_pop)
mn_hawaiian_list
mn_hispanic_list = list(mn_hispanic_pop)
mn_hispanic_list
mn_other_list = list(mn_other_pop)
mn_other_list
print(type(mn_pop_list[0]))

<class 'str'>


In [30]:
#converting string to integer
# print([int(x) for x in mn_pop_list])
mn_pop_int_list = [int(x) for x in mn_pop_list]
mn_caucasian_int_list = [int(x) for x in mn_caucasian_list]
mn_african_american_int_list = [int(x) for x in mn_african_american_list]
mn_native_american_int_list = [int(x) for x in mn_native_american_list]
mn_asian_int_list = [int(x) for x in mn_asian_list]
mn_hawaiian_int_list = [int(x) for x in mn_hawaiian_list]
mn_hispanic_int_list = [int(x) for x in mn_hispanic_list]
mn_other_int_list = [int(x) for x in mn_other_list]


print(type(mn_pop_int_list[0]))

<class 'int'>


In [31]:
#sum
sum_mn_pop = sum(mn_pop_int_list)
sum_mn_caucasian = sum(mn_caucasian_int_list)
sum_mn_african_american = sum(mn_african_american_int_list)
sum_mn_native_american = sum(mn_native_american_int_list)
sum_mn_asian = sum(mn_asian_int_list)
sum_mn_hawaiian = sum(mn_hawaiian_int_list)
sum_mn_hispanic = sum(mn_hispanic_int_list)
sum_mn_other = sum(mn_other_int_list)

#percent
percent_mn_caucasian = sum_mn_caucasian/sum_mn_pop*100
percent_mn_african_american = sum_mn_african_american/sum_mn_pop*100
percent_mn_native_american = sum_mn_native_american/sum_mn_pop*100
percent_mn_asian = sum_mn_asian/sum_mn_pop*100
percent_mn_hawaiian = sum_mn_hawaiian/sum_mn_pop*100
percent_mn_hispanic = sum_mn_hispanic/sum_mn_pop*100
percent_mn_other = sum_mn_other/sum_mn_pop*100

print(sum_mn_pop)
print(sum_mn_caucasian)
print(sum_mn_african_american)
print(sum_mn_native_american)
print(sum_mn_asian)
print(sum_mn_hawaiian)
print(sum_mn_hispanic)
print(sum_mn_other)
print("-----------------")
print(percent_mn_caucasian)
print(percent_mn_african_american)
print(percent_mn_native_american)
print(percent_mn_asian)
print(percent_mn_hawaiian)
print(percent_mn_hispanic)
print(percent_mn_other)

5600166
4422490
354540
48409
273100
1905
307675
192047
-----------------
78.97069479726136
6.330883763088452
0.8644208046690045
4.876641156708569
0.03401684878626812
5.494033569719183
3.429309059767157


In [32]:
#infection rate African American
african_american_infection_percent = (cdc_infections_african_american_total / sum_mn_african_american ) *100
african_american_infection_percent

#infection rate Asian
asian_infection_percent = (cdc_infections_asian_total / sum_mn_asian ) *100
asian_infection_percent

#infection rate Caucasian
caucasian_infection_percent = (cdc_infections_caucasian_total / sum_mn_caucasian ) *100
caucasian_infection_percent

#infection rate hispanic
hispanic_infection_percent = (cdc_infections_hispanic_total / sum_mn_hispanic ) *100
hispanic_infection_percent

#infection rate native american
native_american_infection_percent = (cdc_infections_native_american_total / sum_mn_native_american ) *100
native_american_infection_percent


other_infection_percent = (cdc_infections_other_total / sum_mn_other ) *100
other_infection_percent


percentage_race_infections = [african_american_infection_percent, asian_infection_percent, 
                              caucasian_infection_percent, hispanic_infection_percent, native_american_infection_percent, other_infection_percent]


# for (race, infection_percentage) in zip(race_mn, percentage_race_infections):
#     print(f" {race}: {infection_percentage}")


In [33]:
#rate corrected by population,% (non-whites)
african_american_infection_percent_corrected = african_american_infection_percent / percent_mn_african_american
african_american_infection_percent_corrected
asian_infection_percent_corrected = asian_infection_percent / percent_mn_asian
hispanic_infection_percent_corrected = hispanic_infection_percent / percent_mn_hispanic
native_american_infection_percent_corrected = native_american_infection_percent / percent_mn_native_american
other_infection_percent_corrected = other_infection_percent / percent_mn_other


#rate corrected by population,% (whites)
caucasian_infection_percent_corrected = caucasian_infection_percent / percent_mn_caucasian

## Final Summary table of Infection rate corrected

In [34]:
#dataframe showing infection rate by group (corrected by % population)

race_percent_corrected_df = pd.DataFrame({'% African American':[african_american_infection_percent_corrected],
                     '% Asian':[asian_infection_percent_corrected],
                     '% Caucasian American':[caucasian_infection_percent_corrected],
                     '% Hispanic':[hispanic_infection_percent_corrected],
                     '% Native American':[native_american_infection_percent_corrected],
                     '% Other':[other_infection_percent_corrected] 
                               })

In [35]:
# Declare a list that is to be converted into a column
State = ['Infection rate by race group, %']
race_percent_corrected_df['Minnesota'] = State
race_percent_corrected_df = race_percent_corrected_df.reindex(columns=['Minnesota', '% African American', '% Asian', '% Caucasian American', '% Hispanic',
       '% Native American', '% Other'])
race_percent_corrected_df

,Minnesota,% African American,% Asian,% Caucasian American,% Hispanic,% Native American,% Other
0,"Infection rate by race group, %",1.62469,1.561183,0.094071,0.607261,7.276725,0.834967


## Final Summary table of Infection rate corrected
    a) Caucasian vs Non-caucasian

In [36]:
#Adding a column.
# sum of non-caucasian columns
# Create List of columns
non_caucasian_list= ['% African American', '% Asian',
       '% Hispanic', '% Native American', '% Other']

race_percent_corrected_df['% Non-Caucasian American'] = (race_percent_corrected_df[non_caucasian_list].sum(axis=1)) / 5
race_percent_corrected_df
race_percent_summary_df = race_percent_corrected_df[['Minnesota', '% Caucasian American', '% Non-Caucasian American' ]]
race_percent_summary_df

,Minnesota,% Caucasian American,% Non-Caucasian American
0,"Infection rate by race group, %",0.094071,2.380965


## Hypothesis testing

In [37]:
comprehensive_df.head()


,African American Infections,Asian Infections,Caucasian American Infections,Hispanic Infections,Native American Infections,Other Infections
County,,,,,,
ANOKA,2864.0,1732.0,24501,746.0,73.0,348.0
BELTRAMI,3.0,NaN,2216,NaN,1064.0,33.0
BENTON,104.0,NaN,4164,1.0,NaN,12.0
BLUE EARTH,213.0,30.0,4555,50.0,NaN,21.0
CARLTON,13.0,NaN,1838,NaN,64.0,6.0


In [38]:
mn_pop_race_df = mn_pop_df[['County', 'African American', 'Asian', 'Caucasian', 'Native American', 'Hispanic', 'Other']]
mn_pop_race_df

,County,African American,Asian,Caucasian,Native American,Hispanic,Other
0,ANOKA,23534,16082,283436,1788,16769,12080
1,BECKER,146,209,29290,1938,738,1900
2,BELTRAMI,399,295,33506,8920,1114,2514
3,BENTON,1876,378,35979,49,1086,1108
4,BIG STONE,3,0,4787,3,102,79
...,...,...,...,...,...,...,...
82,RENVILLE,67,108,12741,186,1268,202
83,ROSEAU,124,399,13876,254,223,383
84,SHERBURNE,2658,1206,86342,403,2577,2829
85,STEELE,1225,150,31640,119,2913,657


In [39]:

cdc_census_combined_df = pd.merge(comprehensive_df, mn_pop_race_df, on=['County'], how = "outer")
cdc_census_clean_combined_df = cdc_census_combined_df.fillna(0)
cdc_census_clean_combined_df
# check for na values
# cdc_census_clean_combined_df.isna().sum()

,County,African American Infections,Asian Infections,Caucasian American Infections,Hispanic Infections,Native American Infections,Other Infections,African American,Asian,Caucasian,Native American,Hispanic,Other
0,ANOKA,2864.0,1732.0,24501.0,746.0,73.0,348.0,23534,16082,283436,1788,16769,12080
1,BELTRAMI,3.0,0.0,2216.0,0.0,1064.0,33.0,399,295,33506,8920,1114,2514
2,BENTON,104.0,0.0,4164.0,1.0,0.0,12.0,1876,378,35979,49,1086,1108
3,BLUE EARTH,213.0,30.0,4555.0,50.0,0.0,21.0,2794,1516,58852,139,2579,1473
4,CARLTON,13.0,0.0,1838.0,0.0,64.0,6.0,909,231,31459,1462,638,1010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,LINCOLN,0.0,0.0,0.0,0.0,0.0,0.0,13,31,5394,10,119,74
83,MAHNOMEN,0.0,0.0,0.0,0.0,0.0,0.0,40,19,2468,1963,257,756
84,PIPESTONE,0.0,0.0,0.0,0.0,0.0,0.0,27,71,8007,110,662,286
85,RENVILLE,0.0,0.0,0.0,0.0,0.0,0.0,67,108,12741,186,1268,202


In [40]:
cdc_census_clean_combined_df.columns

Index(['County', 'African American Infections', 'Asian Infections',
       'Caucasian American Infections', 'Hispanic Infections',
       'Native American Infections', 'Other Infections', 'African American',
       'Asian', 'Caucasian', 'Native American', 'Hispanic', 'Other'],
      dtype='object')

In [41]:

# cdc_census_clean_combined_african_american_df = cdc_census_clean_combined_df['African American']
# cdc_census_clean_combined_african_american_df = cdc_census_clean_combined_df['African American'].astype(int)
# cdc_census_clean_combined_df['Asian'] = cdc_census_clean_combined_df['Asian'].astype(int)
# cdc_census_clean_combined_df['Caucasian'] = cdc_census_clean_combined_df['Caucasian'].astype(int)
# cdc_census_clean_combined_df['Native American'] = cdc_census_clean_combined_df['Native American'].astype(int)
# cdc_census_clean_combined_df['Hispanic'] = cdc_census_clean_combined_df['Hispanic'].astype(int)
# cdc_census_clean_combined_df['Other'] = cdc_census_clean_combined_df['Other'].astype(int)

# cdc_census_clean_combined_sum_df = (cdc_census_clean_combined_df['African American'] + cdc_census_clean_combined_df['Asian'] + cdc_census_clean_combined_df['Caucasian'] + cdc_census_clean_combined_df['Native American'] + cdc_census_clean_combined_df['Hispanic'] + cdc_census_clean_combined_df['Other'])
# cdc_census_clean_combined_sum_df

In [42]:
#converting all the rows to integer
column_list = ['African American Infections', 'Asian Infections',
       'Caucasian American Infections', 'Hispanic Infections',
       'Native American Infections', 'Other Infections', 'African American',
       'Asian', 'Caucasian', 'Native American', 'Hispanic', 'Other']

for column in column_list:
    cdc_census_clean_combined_df[column] = cdc_census_clean_combined_df[column].astype(int)


In [43]:
african_american_infection_rate_corrected_df = (((cdc_census_clean_combined_df['African American Infections'] / cdc_census_clean_combined_df['African American']) * 100) / percent_mn_african_american)
african_american_infection_rate_corrected_df

0     1.922264
1     0.118764
2     0.875661
3     1.204173
4     0.225899
        ...   
82    0.000000
83    0.000000
84    0.000000
85    0.000000
86    0.000000
Length: 87, dtype: float64

In [44]:
asian_infection_rate_corrected_df = (((cdc_census_clean_combined_df['Asian Infections'] / cdc_census_clean_combined_df['Asian']) * 100) / percent_mn_asian)
caucasian_american_infection_rate_corrected_df = (((cdc_census_clean_combined_df[ 'Caucasian American Infections'] / cdc_census_clean_combined_df['Caucasian']) * 100) / percent_mn_caucasian)
native_american_infection_rate_corrected_df = (((cdc_census_clean_combined_df['Native American Infections'] / cdc_census_clean_combined_df['Native American']) * 100) / percent_mn_native_american)
hispanic_infection_rate_corrected_df = (((cdc_census_clean_combined_df['Hispanic Infections'] / cdc_census_clean_combined_df['Hispanic']) * 100) / percent_mn_hispanic)
other_infection_rate_corrected_df = (((cdc_census_clean_combined_df['Other Infections'] / cdc_census_clean_combined_df['Other']) * 100) / percent_mn_other)


In [45]:
#SUM OF NON-CACUASIAN DATAFRAME
sum_non_caucasian_corrected_df = african_american_infection_rate_corrected_df + asian_infection_rate_corrected_df + native_american_infection_rate_corrected_df + hispanic_infection_rate_corrected_df + other_infection_rate_corrected_df
sum_non_caucasian_corrected_df

0     10.503625
1     14.300663
2      1.208238
3      2.378572
4      5.463289
        ...    
82     0.000000
83     0.000000
84     0.000000
85     0.000000
86     0.000000
Length: 87, dtype: float64

In [46]:
cdc_census_combined_summary_df = cdc_census_clean_combined_df[['County']]
cdc_census_combined_summary_df

cdc_census_combined_summary_df[  'Non-Caucasian infection rate'  ] = sum_non_caucasian_corrected_df
cdc_census_combined_summary_df[  'Caucasian infection rate'  ] = caucasian_american_infection_rate_corrected_df
cdc_census_combined_summary_df.head(40)

C:\Users\Sung\AppData\Local\Temp\ipykernel_7248\3307306264.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc_census_combined_summary_df[  'Non-Caucasian infection rate'  ] = sum_non_caucasian_corrected_df
C:\Users\Sung\AppData\Local\Temp\ipykernel_7248\3307306264.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc_census_combined_summary_df[  'Caucasian infection rate'  ] = caucasian_american_infection_rate_corrected_df


,County,Non-Caucasian infection rate,Caucasian infection rate
0,ANOKA,10.503625,0.109462
1,BELTRAMI,14.300663,0.083749
2,BENTON,1.208238,0.146553
3,BLUE EARTH,2.378572,0.098008
4,CARLTON,5.463289,0.073983
5,CARVER,2.820130,0.100496
6,CHISAGO,2.142822,0.101728
7,CLAY,7.153111,0.117814
8,DAKOTA,14.680787,0.102391
9,GOODHUE,1.297901,0.122000


In [47]:
# Calculate the mean for infection rate of non-caucasian group
non_caucasian_infection_rate = cdc_census_combined_summary_df['Non-Caucasian infection rate'].fillna(0).to_numpy()
non_caucasian_infection_rate
# non_caucasian_infection_rate = cdc_census_combined_summary_df['Non-Caucasian infection rate'].to_numpy()
# np.where(non_caucasian_infection_rate != np.nan)

array([10.50362462, 14.30066288,  1.20823804,  2.37857172,  5.46328916,
        2.8201296 ,  2.14282203,  7.15311112, 14.68078723,  1.29790115,
       17.07778121,  0.81759781,  0.84696362,  3.13425248,  1.04296665,
        2.76557514,  5.7114803 ,  0.54592448,  4.50968741,  0.69887912,
       12.49304918, 20.03134584,  5.86856584,  5.16871822, 13.90498815,
        6.52840372,  1.34254962, 11.53197141,  1.08907102,  2.39122348,
        1.85712296,  0.06438279, 13.64062427,  0.7396395 ,  8.23301369,
        0.        ,  0.        ,  0.18368752,  0.        ,  0.77640535,
        2.73465983,  0.22295902,  1.02909141,  0.        ,  0.        ,
        0.        ,  0.10315747,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [48]:
# Export the DataFrames to CSV

# non_caucasian_infection_rate.to_csv("output/non_caucasian_infection_rate.csv", index=True)

In [49]:
# cdc_census_combined_df.fillna(0)

In [50]:

# Calculate the mean for infection rate of caucasian group
caucasian_infection_rate = cdc_census_combined_summary_df['Caucasian infection rate'].to_numpy()
caucasian_infection_rate


array([0.10946186, 0.08374931, 0.14655332, 0.09800793, 0.07398346,
       0.10049567, 0.10172812, 0.1178138 , 0.10239085, 0.12200021,
       0.09730171, 0.11180082, 0.10141972, 0.11313336, 0.07644904,
       0.07553675, 0.11638692, 0.08369931, 0.08074626, 0.10448594,
       0.08956722, 0.09331191, 0.11385384, 0.10868979, 0.10428763,
       0.13160771, 0.10669835, 0.10539979, 0.10915416, 0.10776613,
       0.11467863, 0.09501933, 0.08612   , 0.10226204, 0.08026212,
       0.10576744, 0.10373996, 0.11559778, 0.07741361, 0.08498047,
       0.10151112, 0.08049529, 0.1174717 , 0.09019527, 0.10697419,
       0.1101472 , 0.0866756 , 0.10171848, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [51]:
# Calculate Independent (Two Sample) t-test
st.ttest_ind(non_caucasian_infection_rate, caucasian_infection_rate, equal_var=False)

# nan_policy="omit"

Ttest_indResult(statistic=4.840042673223031, pvalue=5.6582378031512245e-06)

In [52]:
#dataframe showing infection rate by group (corrected by % population)

county_list = ['ANOKA', 'BELTRAMI', 'BENTON', 'BLUE EARTH', 'CARLTON', 'CARVER',
       'CHISAGO', 'CLAY', 'DAKOTA', 'GOODHUE', 'HENNEPIN', 'KANDIYOHI',
       'LYON', 'MOWER', 'NICOLLET', 'NOBLES', 'OLMSTED', 'OTTER TAIL',
       'PINE', 'POLK', 'RAMSEY', 'RICE', 'SCOTT', 'SHERBURNE',
       'ST. LOUIS', 'STEARNS', 'STEELE', 'WASHINGTON', 'WINONA', 'WRIGHT',
       'FREEBORN', 'ISANTI', 'BECKER', 'BROWN', 'CASS', 'CROW WING',
       'DODGE', 'DOUGLAS', 'FILLMORE', 'HUBBARD', 'ITASCA', 'LE SUEUR',
       'MCLEOD', 'MEEKER', 'MILLE LACS', 'MORRISON', 'TODD', 'WABASHA',
       'BIG STONE', 'CHIPPEWA', 'WATONWAN', 'WADENA', 'CLEARWATER',
       'COOK', 'COTTONWOOD', 'FARIBAULT', 'GRANT', 'HOUSTON', 'KANABEC',
       'KOOCHICHING', 'LAC QUI PARLE', 'LAKE OF THE WOODS', 'MARSHALL',
       'MARTIN', 'MURRAY', 'NORMAN', 'PENNINGTON', 'POPE', 'RED LAKE',
       'REDWOOD', 'ROCK', 'SIBLEY', 'STEVENS', 'SWIFT', 'TRAVERSE',
       'WASECA', 'WILKIN', 'YELLOW MEDICINE', 'AITKIN', 'JACKSON',
       'KITTSON', 'LAKE', 'LINCOLN', 'MAHNOMEN', 'PIPESTONE', 'RENVILLE',
       'ROSEAU']

cdc_census_combined_allrace_summary_df = pd.DataFrame({'County': county_list,
                     '% African American': african_american_infection_rate_corrected_df,
                     '% Asian': asian_infection_rate_corrected_df,
                     '% Caucasian American': caucasian_american_infection_rate_corrected_df,
                     '% Hispanic': hispanic_infection_rate_corrected_df,
                     '% Native American': native_american_infection_rate_corrected_df,
                     '% Other': other_infection_rate_corrected_df 
                               })

cdc_census_combined_allrace_summary_df

,County,% African American,% Asian,% Caucasian American,% Hispanic,% Native American,% Other
0,ANOKA,1.922264,2.208447,0.109462,0.80973,4.723133,0.840051
1,BELTRAMI,0.118764,0.000000,0.083749,0.00000,13.799125,0.382774
2,BENTON,0.875661,0.000000,0.146553,0.01676,0.000000,0.315817
3,BLUE EARTH,1.204173,0.405790,0.098008,0.35288,0.000000,0.415729
4,CARLTON,0.225899,0.000000,0.073983,0.00000,5.064160,0.173230
...,...,...,...,...,...,...,...
82,LINCOLN,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
83,MAHNOMEN,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
84,PIPESTONE,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
85,RENVILLE,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000


In [ ]:
#declare groups
group0 = african_american_infection_rate_corrected_df.fillna(0)
group1 = asian_infection_rate_corrected_df.fillna(0)
group2 = caucasian_american_infection_rate_corrected_df.fillna(0)
group3 = hispanic_infection_rate_corrected_df.fillna(0)
group4 = native_american_infection_rate_corrected_df.fillna(0)
group5 = other_infection_rate_corrected_df.fillna(0)

# group1 = asian_infection_rate_corrected_df.fillna(0).to_numpy()

In [ ]:
#ANOVA test
# Perform the ANOVA
st.f_oneway(group0, group1, group2, group3, group4, group5)

In [ ]:
#population by race-Summary
cdc_byrace_df = cdc_df['Race'].value_counts()
cdc_byrace_df

### Merge the datasets

In [ ]:
# Edited merge_df for nan infections to be 0
merged_df = pd.merge(pop_df, poverty_df, on=['County', 'Race'])
merged_df = pd.merge(merged_df, cdc_df, on=['County', 'Race'], how = "left")
merged_df["Infections"] = merged_df["Infections"].fillna(0)
merged_df

In [ ]:
len(merged_df)

In [ ]:
# Convert all non-caucasian races to "non-caucasian"
#non_caucasian = ['African American', 'Hispanic', 'Asian', 'Native American', 'Hawaiian', 'Other']
merged_df['Multi-Racial']=["Non-Caucasian" if (x=="African American" or x=="Asian" or x=="Hispanic" or x=="Native American"\
    or x=="Hawaiian" or x=="Other") else "Caucasian" for x in merged_df['Race']]
merged_df

In [ ]:
# Convert all numerical values in merged_df from strings to int
merged_df["Population"] = merged_df["Population"].astype(int)
merged_df["Poverty"] = merged_df["Poverty"].astype(int)
merged_df["Infections"] = merged_df["Infections"].astype(int)

# Calculate poverty and infection rates
poverty_rate = []
infection_rate = []

i = 0
while i < len(merged_df):
    if merged_df["Population"][i] == 0:
        poverty_rate.append(0)
        infection_rate.append(0)
    else: 
        poverty_rate.append( (merged_df["Poverty"][i] / merged_df["Population"][i])   * 100  )
        infection_rate.append(merged_df["Infections"][i] / merged_df["Population"][i]  * 100  )
    i += 1
    
merged_df["Poverty Rate"] = poverty_rate
merged_df["Infection Rate"] = infection_rate


merged_df

# merged_df[merged_df["County"] == "CROW WING"]  


In [ ]:
merged_sorted_df = merged_df.sort_values(['case_month', 'County'])
merged_sorted_df

In [ ]:
merged_sorted2_df = merged_sorted_df[['County', 'Multi-Racial', 'Poverty Rate',]]
merged_sorted2_df.head(50)


# poverty_caucasian_group_df = poverty_caucasian_df[['County', 'Race', 'Poverty Rate', ]]
# poverty_caucasian_group_df.sum()
# pop_gb = pop_df.groupby(['County', 'Race'])

# asian_poverty_rate_corrected_df = (((cdc_census_clean_combined_df['Asian Infections'] / cdc_census_clean_combined_df['Asian']) * 100) / percent_mn_asian)
# caucasian_american_poverty_rate_corrected_df = (((cdc_census_clean_combined_df[ 'Caucasian American Infections'] / cdc_census_clean_combined_df['Caucasian']) * 100) / percent_mn_caucasian)
# native_american_poverty_rate_corrected_df = (((cdc_census_clean_combined_df['Native American Infections'] / cdc_census_clean_combined_df['Native American']) * 100) / percent_mn_native_american)
# hispanic_poverty_rate_corrected_df = (((cdc_census_clean_combined_df['Hispanic Infections'] / cdc_census_clean_combined_df['Hispanic']) * 100) / percent_mn_hispanic)
# other_poverty_rate_corrected_df = (((cdc_census_clean_combined_df['Other Infections'] / cdc_census_clean_combined_df['Other']) * 100) / percent_mn_other)


# poverty_rate_corrected = merged_sorted2_df['Poverty Rate'] / 

In [ ]:
poverty_caucasian_df = merged_sorted2_df[   merged_sorted2_df["Multi-Racial"] == 'Caucasian']
poverty_caucasian_df

In [ ]:
poverty_rate_caucasian_df = poverty_caucasian_df[['Multi-Racial', 'Poverty Rate']]
poverty_rate_caucasian_df

In [ ]:
poverty_non_caucasian_df = merged_sorted2_df[   merged_sorted2_df["Multi-Racial"] == 'Non-Caucasian']
poverty_non_caucasian_df

In [ ]:
poverty_rate_non_caucasian_df = poverty_non_caucasian_df[['Multi-Racial', 'Poverty Rate']]
poverty_rate_non_caucasian_df

In [ ]:
# Calculate the mean for poverty rate of non-caucasian group
non_caucasian_poverty_rate = poverty_rate_non_caucasian_df['Poverty Rate'].fillna(0)
non_caucasian_poverty_rate.mean()
# non_caucasian_infection_rate = cdc_census_combined_summary_df['Non-Caucasian infection rate'].to_numpy()
# np.where(non_caucasian_infection_rate != np.nan)


In [ ]:
# Calculate the mean for infection rate of caucasian group
caucasian_poverty_rate = poverty_rate_caucasian_df['Poverty Rate'].fillna(0)
caucasian_poverty_rate.mean()

In [ ]:
# Calculate Independent (Two Sample) t-test
st.ttest_ind(non_caucasian_poverty_rate, caucasian_poverty_rate, equal_var=False)


# -------------------------------------

### Coordinates for heat map -- if time permits

In [ ]:
# Get the coordinates of counties in MN
url = "https://en.wikipedia.org/wiki/User:Michael_J/County_table"

table = pd.read_html(url)
df = table[0]
Counties = df.loc[df['State'] == "MN", : ]
counties_df = Counties[['County [2]', 'Latitude', 'Longitude']]
counties_df#.to_csv("../Project_1/county_geo.csv")  

# -------------------------------------**Sung** Work Ends Here-----------------------------------